#Step 1. Pytorch Installation
**Installing base-framework Pytorch version 1.9 in Google's coolab.**

In [1]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:16:04tcmalloc: large alloc 1147494400 bytes == 0x56110b670000 @  0x7f33c94bb615 0x5610d286c02c 0x5610d294c17a 0x5610d286ee4d 0x5610d2960c0d 0x5610d28e30d8 0x5610d28ddc35 0x5610d287073a 0x5610d28e2f40 0x5610d28ddc35 0x5610d287073a 0x5610d28df93b 0x5610d2961a56 0x5610d28defb3 0x5610d2961a56 0x5610d28defb3 0x5610d2961a56 0x5610d28defb3 0x5610d2870b99 0x5610d28b3e79 0x5610d286f7b2 0x5610d28e2e65 0x5610d28ddc35 0x5610d287073a 0x5610d28df93b 0x5610d28ddc35 0x5610d287073a 0x5610d28deb0e 0x5610d287065a 0x5610d28ded67 0x5610d28ddc35
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:59tcmalloc: large alloc 1434370048 bytes == 0x56114fcc6000 @  0x7f33c94bb615 0x5610d286c02c 0x5610d294c17a 0x5610d286ee4d 0x5610d2960c0d 0x5610d28e30d8 0x5610d28ddc35 0x5610d287073a 0x5610d28e2f40 0x5610d28ddc35 0x5610d287073a 0x5610d28df93b 0x5610d2961a56 0x5610d28def

# Step 2. Library installations
**Installing necessary libraries for running BERT models**

In [2]:
!pip install transformers pandas numpy

     |████████████████████████████████| 2.6 MB 7.5 MB/s 
     |████████████████████████████████| 3.3 MB 50.3 MB/s 
     |████████████████████████████████| 895 kB 59.2 MB/s 
     |████████████████████████████████| 636 kB 51.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Step 3. Importing necessary libraries


1.   **Pytorch as base-framework.**
2.   **Also, Tokenizer and Model from transformers.**
3.   **Pandas**



In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
import pandas as pd

# Step 4. Importing local files in Google's Colab

In [6]:
from google.colab import files
uploaded = files.upload()

Saving BERT_sents_labelled.csv to BERT_sents_labelled.csv


In [7]:
import io
reviews = pd.read_csv(io.BytesIO(uploaded['BERT_sents_labelled.csv']))

# Step 5. Loading the dataframe "reviews"

In [8]:
reviews

,spk_rc,interview,spk_no,Cand_Rcrt_No,sentiment,phases
0,spk_0,So I'm steveneu chue. I currently doing a mast...,spk_0_0,Cand_014633,Positive,AboutSkillsAndEducation
1,spk_0,I would rate myself around seven for most of t...,spk_0_1,Cand_014633,Neutral,AboutSkillsAndEducation
2,spk_0,My first strength is I am very determined. My ...,spk_0_2,Cand_014633,Neutral,Behavioral_Question
3,spk_0,I think my top three witnesses. I tend to get ...,spk_0_3,Cand_014633,Positive,Behavioral_Question
4,spk_0,"Um, just about the internships. Would it be mo...",spk_0_4,Cand_014633,Neutral,AboutInternship
...,...,...,...,...,...,...
222,spk_1,"Yeah, because I was introduced by some areas, ...",spk_1_8,Cand_053949,Negative,AboutInternship
223,spk_1,I have noticed that the government has announc...,spk_1_9,Cand_053949,Neutral,Conclusion.
224,spk_1,"inal, like previous experience is the candidat...",spk_1_10,Cand_053949,Neutral,AboutInternship
225,spk_1,"Does the programme includes, like, resume modi...",spk_1_11,Cand_053949,Neutral,AboutInternship


# Step 6. Marking the "reviews" sentiments
**Defining Sentiment score function for marking the transcripts according to their sentiments**

In [9]:
def sentiment_score(interview):
   tokens = tokenizer.encode(interview, return_tensors='pt')
   result = model(tokens)
   return int(torch.argmax(result.logits))+1

#Step 7. Sentiment Scores Insertion in "reviews"
**Inserting the generated sentiment scores in the "sentiment" column of dataframe "reviews"**

In [10]:
reviews['sentiment_scores'] = reviews['interview'].apply(lambda x: sentiment_score(x[:512]))
reviews

,spk_rc,interview,spk_no,Cand_Rcrt_No,sentiment,phases,sentiment_scores
0,spk_0,So I'm steveneu chue. I currently doing a mast...,spk_0_0,Cand_014633,Positive,AboutSkillsAndEducation,4
1,spk_0,I would rate myself around seven for most of t...,spk_0_1,Cand_014633,Neutral,AboutSkillsAndEducation,3
2,spk_0,My first strength is I am very determined. My ...,spk_0_2,Cand_014633,Neutral,Behavioral_Question,3
3,spk_0,I think my top three witnesses. I tend to get ...,spk_0_3,Cand_014633,Positive,Behavioral_Question,4
4,spk_0,"Um, just about the internships. Would it be mo...",spk_0_4,Cand_014633,Neutral,AboutInternship,3
...,...,...,...,...,...,...,...
222,spk_1,"Yeah, because I was introduced by some areas, ...",spk_1_8,Cand_053949,Negative,AboutInternship,2
223,spk_1,I have noticed that the government has announc...,spk_1_9,Cand_053949,Neutral,Conclusion.,3
224,spk_1,"inal, like previous experience is the candidat...",spk_1_10,Cand_053949,Neutral,AboutInternship,3
225,spk_1,"Does the programme includes, like, resume modi...",spk_1_11,Cand_053949,Neutral,AboutInternship,3


# Step 8. Counting sentiments

In [ ]:
sents_vals = reviews['sentiment'].value_counts()
sents_vals

3    103
4     48
5     29
2     24
1     23
Name: sentiment, dtype: int64

# Step 9. Dataset Labelling 
**Labelling the dataset by "sentiment" scores e.g.**
1.   **1,2  scores for Negative sentiment.**
2.   **3 when Neutral sentiment**
3.   **4,5 for Positive sentiment.**


 
 

In [ ]:
rs = reviews['sentiment']
count, count2, count3 = 0, 0, 0 
for i, value in rs.items():
      if value<=2:
         count+=1
      elif value==3:
         count2+=1
      else:
        count3+=1
print("Scores using BERT:\n Positive: {}\n Neutral: {}\n Negative: {}".format(count3, count2, count))

Scores using BERT:
 Positive: 77
 Neutral: 103
 Negative: 47


In [ ]:
reviews['sentiment'].mask(reviews['sentiment'] == 5, 'Positive', inplace=True)

In [ ]:
reviews

,spk_rc,interview,spk_no,Cand_Rcrt_No,sentiment
0,spk_0,So I'm steveneu chue. I currently doing a mast...,spk_0_0,Cand_014633,Positive
1,spk_0,I would rate myself around seven for most of t...,spk_0_1,Cand_014633,Neutral
2,spk_0,My first strength is I am very determined. My ...,spk_0_2,Cand_014633,Neutral
3,spk_0,I think my top three witnesses. I tend to get ...,spk_0_3,Cand_014633,Positive
4,spk_0,"Um, just about the internships. Would it be mo...",spk_0_4,Cand_014633,Neutral
...,...,...,...,...,...
222,spk_1,"Yeah, because I was introduced by some areas, ...",spk_1_8,Cand_053949,Negative
223,spk_1,I have noticed that the government has announc...,spk_1_9,Cand_053949,Neutral
224,spk_1,"inal, like previous experience is the candidat...",spk_1_10,Cand_053949,Neutral
225,spk_1,"Does the programme includes, like, resume modi...",spk_1_11,Cand_053949,Neutral


# Step 10. Download the labelled-dataset

In [11]:
from google.colab import files
reviews.to_csv('BERT_sents_with_scores.csv', index=False) 
files.download('BERT_sents_with_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>